# **Describe the image**

In [ ]:
!pip install transformers

In [ ]:
from huggingface_hub import InferenceClient
from kaggle_secrets import UserSecretsClient
 
#connect to HF
user_secrets = UserSecretsClient()
HF_API = user_secrets.get_secret("HuggingFaceAPI")

client = InferenceClient(api_key="" + HF_API)

messages = [
	{
		"role": "user",
		"content": [
			{
				"type": "text", #system prompt
				"text": "Describe the overall scene, then describe vividly the main subjects and actions with an emphasis on the emotion in 3 sentences."
			},
			{
				"type": "image_url",
				"image_url": { # example image
					"url": "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Anatomy_of_a_Sunset-2.jpg/1200px-Anatomy_of_a_Sunset-2.jpg"
				}
			}
		]
	}
]

stream = client.chat.completions.create(
    model="meta-llama/Llama-3.2-11B-Vision-Instruct", 
	messages=messages, 
	stream=True
)

res = ""

for chunk in stream:
    res += (chunk.choices[0].delta.content)

print(res)

# **Use an LLM to analyze description based on spotify filters**

In [ ]:
import requests
import re

API_URL = "https://api-inference.huggingface.co/models/meta-llama/Llama-3.2-3B-Instruct"
headers = {"Authorization": "Bearer " + HF_API}

# Extract the part after the last assistant header
def extract_last_response(raw_response):
    # Regex to find the last assistant section
    match = re.findall(
        r"<\|start_header_id\|>assistant<\|end_header_id\|>\s*(.*?)(?=<\|start_header_id\|>user<\|end_header_id\|>|\Z)",
        raw_response,
        re.DOTALL,
    )
    if match:
        # Return the last match and clean up any unwanted markers
        response = match[-1].strip()
        return response.replace("<|eot_id|>", "").strip()
    return "Error: Unable to extract response"

def query(payload):
    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()  # Return the JSON response
    except requests.exceptions.RequestException as e:
        print(f"API Error: {e}")
        return {"generated_text": ""}

def format_prompt(sp, exUsrInpt, exResponse, p):
    return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

    {sp}<|eot_id|><|start_header_id|>user<|end_header_id|>

    {exUsrInpt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
    
    {exResponse}<|eot_id|><|start_header_id|>user<|end_header_id|>

    {p}<|eot_id|><|start_header_id|>assistant<|end_header_id|>""" 

# Task 2 settings
settings2 = {"temperature": 1, "top_k": 50, "top_p": 0.3, "max_new_tokens": 200}

# Task 2 inputs
system_prompt2 = """You are an expert at classifying a prompt for song lookup in Spotify, here is the criteria to evaluate on: 
min_danceability
max_danceability
min_energy
max_energy
min_instrumentalness
max_instrumentalness
min_loudness
max_loudness
min_tempo
max_tempo
min_valence
max_valence
This is an example:"""

exP = "The sun sets over the ocean, golden light piercing clouds, evoking serenity and bittersweet wonder."

exR2 = """min_danceability: 0.1,
    max_danceability: 0.3,
    min_energy: 0.2,
    max_energy: 0.5,
    min_instrumentalness: 0.6,
    max_instrumentalness: 1.0,
    min_loudness: 0.1,
    max_loudness: 0.4,
    min_tempo: 0.3,
    max_tempo: 0.6,
    min_valence: 0.2,
    max_valence: 0.4"""

prompt2 = format_prompt(system_prompt2, exP, exR2, res)

# Task 2 API call
response2 = query({"inputs": prompt2, "parameters": settings2})

# Task 2 result
raw_task2_response = response2[0]["generated_text"] 
task2_result = extract_last_response(raw_task2_response)  # Now extracting the response content
print("Task 2 Result:")
print(task2_result)

# **Get the songs from spotify**

In [ ]:
!pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

key = user_secrets.get_secret("SpotifyAPIKey")
ID = user_secrets.get_secret("SpotifyID")

#Spotify connection
client_credentials_manager = SpotifyClientCredentials(client_id="" + ID, client_secret="" + key)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Get the params
params = {}
for line in task2_result: 
    line.strip
    filter_name = re.findall(r"^(\w+):", line) #get filter
    filter_val = float(re.findall(r": (\d.\d+),", line)) # get filter value
    params[filter_name] = filter_val
    
    
    
# Search using query
results = sp.recommendations(**params)
    
artists = []
songs = ""

# get the results, avoiding repetitive artists
for track in results['tracks']['items']:
    if len(artists) == 5:
        break
    elif track['artists'][0]['name'] in artists:
        pass
    else:
        artists.append(track['artists'][0]['name'])
        songs += (f"Track: {track['name']}, Artist: {track['artists'][0]['name']} \n")

# print results
print(songs)